In [379]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt     

In [380]:
df = pd.read_csv('course_lead_scoring.csv')

In [381]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [382]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [383]:
df.columns = df.columns.str.lower().str.replace(' ', '_')
string_columns = list(df.dtypes[df.dtypes == 'object'].index)

for col in string_columns:
    df[col] = df[col].str.lower().str.replace(' ', '_')

In [384]:
categorical = ['lead_source', 'industry', 'employment_status', 'location']
numerical = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score', 'converted']

In [385]:
df[categorical].isnull().sum()

lead_source          128
industry             134
employment_status    100
location              63
dtype: int64

In [386]:
df[categorical] = df[categorical].fillna('NA')

In [387]:
df[numerical].isnull().sum()

number_of_courses_viewed      0
annual_income               181
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [388]:
df['annual_income'] = df['annual_income'].fillna(0.0)

In [389]:
df[numerical].isnull().sum()

number_of_courses_viewed    0
annual_income               0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

In [390]:
df['industry'].mode()

0    retail
Name: industry, dtype: object

**Q1: What is the most frequent observation (mode) for the column industry?**

**A: retail**

In [391]:
correlations_list = [
    df['interaction_count'].corr(df['lead_score']),
    df['number_of_courses_viewed'].corr(df['lead_score']),
    df['number_of_courses_viewed'].corr(df['interaction_count']),
    df['annual_income'].corr(df['interaction_count'])
]
correlations_list

[np.float64(0.009888182496913084),
 np.float64(-0.004878998354681257),
 np.float64(-0.023565222882888117),
 np.float64(0.02703647240481436)]

In [392]:
max_correlation = max(correlations_list)
max_correlation

np.float64(0.02703647240481436)

**Q2: What are the two features that have the biggest correlation?**

**A: annual_income and interaction_count**

In [393]:
from sklearn.model_selection import train_test_split

In [394]:
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_train_full, test_size=0.25, random_state=42)

In [395]:
y_train = df_train.converted.values
y_val = df_val.converted.values

In [396]:
del df_train['converted']
del df_val['converted']

In [397]:
from sklearn.metrics import mutual_info_score

In [398]:
def calculate_mi(series):
    return mutual_info_score(series, y_train)

df_mi = df_train[categorical].apply(calculate_mi)
df_mi = df_mi.sort_values(ascending=False).to_frame(name='MI')
round(df_mi,2)


df_mi

,MI
lead_source,0.035396
employment_status,0.012938
industry,0.011575
location,0.004464


**Q3: Which of these variables has the biggest mutual information score?**

**A: lead_source**

In [399]:
from sklearn.feature_extraction import DictVectorizer

In [400]:
train_dict = df_train.to_dict(orient='records')

In [401]:
dv = DictVectorizer(sparse=False)
dv.fit(train_dict)

,dtype,<class 'numpy.float64'>
,separator,'='
,sparse,False
,sort,True


In [402]:
X_train = dv.transform(train_dict)
X_train.shape

(876, 31)

In [403]:
dv.get_feature_names_out()

array(['annual_income', 'employment_status=NA',
       'employment_status=employed', 'employment_status=self_employed',
       'employment_status=student', 'employment_status=unemployed',
       'industry=NA', 'industry=education', 'industry=finance',
       'industry=healthcare', 'industry=manufacturing', 'industry=other',
       'industry=retail', 'industry=technology', 'interaction_count',
       'lead_score', 'lead_source=NA', 'lead_source=events',
       'lead_source=organic_search', 'lead_source=paid_ads',
       'lead_source=referral', 'lead_source=social_media', 'location=NA',
       'location=africa', 'location=asia', 'location=australia',
       'location=europe', 'location=middle_east',
       'location=north_america', 'location=south_america',
       'number_of_courses_viewed'], dtype=object)

In [404]:
from sklearn.linear_model import LogisticRegression

In [405]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [406]:
val_dict = df_val.to_dict(orient='records')
X_val = dv.transform(val_dict)

In [407]:
y_pred = model.predict_proba(X_val)[:, 1]
y_pred

array([0.61192163, 0.79982617, 0.53021344, 0.47131479, 0.57066132,
       0.44227169, 0.87127669, 0.84883115, 0.83290037, 0.61497801,
       0.54968027, 0.78153088, 0.69039786, 0.77017122, 0.5265944 ,
       0.91706425, 0.53170635, 0.42123049, 0.30146455, 0.84881583,
       0.79488653, 0.73670375, 0.44527211, 0.64838383, 0.4176882 ,
       0.75393418, 0.90166116, 0.33903049, 0.43181431, 0.9680681 ,
       0.92018714, 0.37487988, 0.652301  , 0.90650057, 0.75164117,
       0.64202121, 0.82250075, 0.83375553, 0.659116  , 0.30978853,
       0.78942264, 0.35546366, 0.96517758, 0.63389304, 0.51274195,
       0.53230533, 0.82287785, 0.744074  , 0.73452313, 0.68955217,
       0.46964443, 0.84539252, 0.55635243, 0.92637871, 0.65258021,
       0.61526273, 0.63816995, 0.28304018, 0.48049824, 0.57890618,
       0.35497342, 0.62175051, 0.38960778, 0.61156056, 0.85304278,
       0.75430136, 0.89185954, 0.71946459, 0.95387623, 0.89209517,
       0.75277088, 0.33850139, 0.61376593, 0.51622275, 0.64088

In [408]:
converted_decision = (y_pred >= 0.5)

In [409]:
accuracy = (y_val == converted_decision).mean()
accuracy.round(2)

np.float64(0.7)

**Q4: What accuracy did you get?**

**A: 0.7**

In [410]:
categorical = ['industry','employment_status','location']
numerical = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']

In [411]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [412]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [413]:
y_pred = model.predict_proba(X_val)[:, 1]
converted_decision = (y_pred >= 0.5)
accuracy_no_lead = (y_val == converted_decision).mean()
accuracy_no_lead

np.float64(0.7030716723549488)

In [414]:
accuracy_no_lead_diff = accuracy - accuracy_no_lead
accuracy_no_lead_diff

np.float64(-0.0034129692832765013)

In [415]:
categorical = ['lead_source','employment_status','location']
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [416]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict_proba(X_val)[:, 1]
converted_decision = (y_pred >= 0.5)
accuracy_no_ind = (y_val == converted_decision).mean()
accuracy_no_ind


np.float64(0.6996587030716723)

In [417]:
accuracy_no_ind_diff = accuracy - accuracy_no_ind
accuracy_no_ind_diff

np.float64(0.0)

In [418]:
categorical = ['lead_source','industry','location']

dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict_proba(X_val)[:, 1]
converted_decision = (y_pred >= 0.5)
accuracy_no_emp = (y_val == converted_decision).mean()
accuracy_no_emp

np.float64(0.6962457337883959)

In [419]:
accuracy_no_emp_diff = accuracy - accuracy_no_emp
accuracy_no_emp_diff

np.float64(0.0034129692832763903)

**Q5: Which of following feature has the smallest difference?**

**A: lead_score**

In [420]:
categorical = ['lead_source','industry','employment_status','location']

dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [421]:
for C in [0.01, 0.1, 1, 10, 100]:

    model = LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict_proba(X_val)[:, 1]
    churn_decision = (y_pred >= 0.5)
    accuracy = (y_val == churn_decision).mean()
    
    print(C, accuracy)

0.01 0.6996587030716723
0.1 0.6996587030716723
1 0.6996587030716723
10 0.6996587030716723
100 0.6996587030716723


**Q6: Which of these C leads to the best accuracy on the validation set?**

**A: 0.01**